In [1]:
import os

import utils

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

# Dataset

- Performance of EPFL Students on the courses 

- 6562 students
- 36 courses
- 7 semesters
- 17490 grades

In [2]:
subset = { 'EE-100', 'EE-102', 'PHYS-101', 'PHYS-106', 'MATH-111', 'MATH-150', 'CS-112', 'CIVIL-123', 'CS-111', 'CIVIL-122', 'ENV-140' }
print('NUMBER OF COURSES:', len(subset))

(data, courses) = utils.getSQLData(subset)
data.head(10)

NUMBER OF COURSES: 11


,StudentID,CourseID,ExamDate,Grade,Result
0,4913,9,2013-07-01,3.5,0
1,4983,9,2013-07-01,3.5,0
2,4629,9,2013-07-01,2.5,0
3,4630,9,2013-07-01,1.0,0
4,4631,9,2013-07-01,3.0,0
5,4984,9,2013-07-01,6.0,1
6,4985,9,2013-07-01,4.0,1
7,2201,9,2013-07-01,4.0,1
8,703,9,2013-07-01,1.5,0
9,4632,9,2013-07-01,2.5,0


# Fitting the LFA model

In [3]:
from model import *

model = ModelLFA()
model.fit(data[['StudentID', 'CourseID', 'ExamDate', 'Result']].values)

In [4]:
l = len(subset)
activities = [ a for a in model.activities.values() if courses[a.id] in subset ]

utils.generateGraph('G1', activities, courses, lambda a,b,i: (1e-2 * a.activityListC[b]), 'yellow')
utils.generateGraph('G2', activities, courses, lambda a,b,i: (25e-1 * a.params[i]), 'red')

In [5]:
s = { 'CS-111', 'CIVIL-123', 'PHYS-106' }

for a in model.activities.values():
    if courses[a.id] in s:
        print('<<<', courses[a.id], '>>>')
        for i, b in enumerate(a.activityList):
            if courses[b] in s:
                print('\t', courses[b], '\t', a.params[i])
        print('\t', 'beta', '\t\t', a.params[-1])
        print('-----------------------------------------------')

<<< PHYS-106 >>>
	 CS-111 	 -0.490722498907
	 CIVIL-123 	 0.0546248926852
	 beta 		 -0.254727839392
-----------------------------------------------
<<< CS-111 >>>
	 PHYS-106 	 0.238293403694
	 beta 		 0.762709979149
-----------------------------------------------
<<< CIVIL-123 >>>
	 PHYS-106 	 3.01727456526
	 CS-111 	 7.59255795009
	 beta 		 -0.405257935966
-----------------------------------------------


# Testing Optimizer with simulations

In [9]:
from simulation import * 
from optimizer import *

nA = 10
nAS = 5
nS = 100

s = SimulationLFA(11, model)

o = RandomOptimizer(nA=10)
s.runWithOptimizer(o, nAS, nS)

print([ (e.student.id, e.activity.id, e.result, e.state) for e in model.events if e.student.id == 0 ])
print('NUM EVENTS', len(model.events))
print(s.evaluateOptimality())


[(0, 5, 1, {}), (0, 8, 1, {5: 1}), (0, 3, 1, {8: 1, 5: 1}), (0, 2, 1, {8: 1, 3: 1, 5: 1}), (0, 0, 1, {8: 1, 2: 1, 3: 1, 5: 1})]
NUM EVENTS 500


NameError: name 'sim' is not defined